## Hacking PubMedHealth API: JSON
### 1) http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmedhealth& term=heart+disease&retmax=10000&usehistory=y&retmode=json
#### http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?
#### db = pubmedhealth
#### term = heart+disease
#### retmax = 10000
#### usehistory = y
#### retmode = json
### 2) http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmedhealth&retmode=json&id
####  http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?
#### id


In [1]:
import urllib2
import json

In [2]:
url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmedhealth&\
       term=heart+disease&retmax=10000&usehistory=y&retmode=json'
parsed = json.load(urllib2.urlopen(url))
#print json.dumps(parsed, indent=4)

In [3]:
counts = parsed['esearchresult']['count']
retmax = parsed['esearchresult']['retmax']
ids = parsed['esearchresult']['idlist']
print counts, retmax

5143 5143


In [7]:
documents = []
for i, id_ in enumerate(ids):
    dict_ = {}
    uid = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmedhealth&retmode=json&id=%s' %id_
    puid = json.load(urllib2.urlopen(uid))
    #print json.dumps(puid, indent=4)
    summary = json.dumps(puid['result'][id_]['teaser'])
    dict_['url'] = uid
    dict_['summary'] = summary.strip('"')
    documents.append(dict_)
    if i < (int(retmax) - 1):
        if i%500 == 0:
            print 'Number of processed articles %d' % i
    else:
        print 'Downloaded %d articles' % len(documents)

Number of processed articles 0
Number of processed articles 500
Number of processed articles 1000
Number of processed articles 1500
Number of processed articles 2000
Number of processed articles 2500
Number of processed articles 3000
Number of processed articles 3500
Number of processed articles 4000
Number of processed articles 4500
Number of processed articles 5000
Downloaded 5143 articles


### Inserting data into MongoDB:
#### db = pubmedhealth
#### collection = heart
#### document = {url, summary}

In [8]:
from pymongo import MongoClient

In [9]:
client = MongoClient()
db = client.pubmedhealth
heart_collection = db.heart
for document in documents:
    heart_collection.save(document)

In [10]:
heart_collection.count()

5143

In [13]:
db.collection_names()

[u'heart', u'system.indexes']